!pip3 uninstall -y torch torchvision

!pip3 install torch torchvision

In [1]:
import torch

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [5]:
import keras
print(keras.__version__)

3.7.0


In [7]:
import numpy as np
import keras
from keras import layers
# from tensorflow import data as tf_data
# import matplotlib.pyplot as plt

In [204]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape: ", y_test.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
y_train shape:  (10000, 10)


In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.7701 - loss: 0.7414 - val_accuracy: 0.9777 - val_loss: 0.0811
Epoch 2/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9644 - loss: 0.1181 - val_accuracy: 0.9830 - val_loss: 0.0581
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.9719 - loss: 0.0899 - val_accuracy: 0.9878 - val_loss: 0.0461
Epoch 4/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9754 - loss: 0.0769 - val_accuracy: 0.9872 - val_loss: 0.0423
Epoch 5/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9816 - loss: 0.0625 - val_accuracy: 0.9892 - val_loss: 0.0399
Epoch 6/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9830 - loss: 0.0540 - val_accuracy: 0.9900 - val_loss: 0.0370
Epoch 7/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.9825 - loss: 0.0542 - val_accuracy: 0.9913 - val_loss: 0.0332
Epoch 8/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.9853 - loss: 0.0473 - va

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.0244517270475626
Test accuracy: 0.9915000200271606


In [ ]:
# split it like above

# do the shaping if necessary
# change the type of the columns that are characters to one-hot-encodings or something like that

# make a model with one layer
# run the model and see what happens

In [33]:
# import the data
import pandas as pd

df = pd.read_csv('data\\online_shoppers_intention.csv')

print(df)

# month, visitor_type, weekend, and revenue need to be converted
# needs to be a numpy array befor being provided as input, so need to make everything encoded (we can try both one-hot and numerical)

#from numpy import genfromtxt
#my_data = genfromtxt('data\\online_shoppers_intention.csv', delimiter=',')

#print(my_data)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [37]:
df['total_duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['proportion_of_duration'] = df['ProductRelated_Duration'] / df['total_duration']

In [39]:
print(df)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [43]:
df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

,BounceRates,ExitRates,PageValues,SpecialDay,Month,Region,TrafficType,VisitorType,Weekend,Revenue,total_duration,proportion_of_duration
0,0.200000,0.200000,0.000000,0.0,Feb,1,1,Returning_Visitor,False,False,0.000000,NaN
1,0.000000,0.100000,0.000000,0.0,Feb,1,2,Returning_Visitor,False,False,64.000000,1.000000
2,0.200000,0.200000,0.000000,0.0,Feb,9,3,Returning_Visitor,False,False,0.000000,NaN
3,0.050000,0.140000,0.000000,0.0,Feb,2,4,Returning_Visitor,False,False,2.666667,1.000000
4,0.020000,0.050000,0.000000,0.0,Feb,1,4,Returning_Visitor,True,False,627.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.007143,0.029031,12.241717,0.0,Dec,1,1,Returning_Visitor,True,False,1928.791667,0.924823
12326,0.000000,0.021333,0.000000,0.0,Nov,1,8,Returning_Visitor,True,False,465.750000,1.000000
12327,0.083333,0.086667,0.000000,0.0,Nov,1,13,Returning_Visitor,True,False,184.250000,1.000000
12328,0.000000,0.021053,0.000000,0.0,Nov,3,11,Returning_Visitor,False,False,421.000000,0.821853


In [45]:
# df['Month'].replace(['
print(df['Month'].unique())

['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec']


In [51]:
df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\727251348.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],


In [53]:
print(df['Month'].unique())

[ 2  3  5 10  6  7  8 11  9 12]


In [55]:
print(df['VisitorType'].unique())

['Returning_Visitor' 'New_Visitor' 'Other']


In [57]:
df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\656729720.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\656729720.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

In [59]:
print(df)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [61]:
df_copy = df.copy(deep=True)

In [63]:
print(df_copy)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [67]:
df_copy['Weekend'].replace([False, True], [0,1], inplace=True)
print(df_copy)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\1718163399.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Weekend'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\1718163399.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['We

In [69]:
df_copy['Revenue'].replace([False, True], [0,1], inplace=True)
print(df_copy)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\4047257151.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Revenue'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_26592\4047257151.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['Re

In [71]:
df_copy = df_copy.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

print(df_copy)

       BounceRates  ExitRates  PageValues  SpecialDay  Month  Region  \
0         0.200000   0.200000    0.000000         0.0      2       1   
1         0.000000   0.100000    0.000000         0.0      2       1   
2         0.200000   0.200000    0.000000         0.0      2       9   
3         0.050000   0.140000    0.000000         0.0      2       2   
4         0.020000   0.050000    0.000000         0.0      2       1   
...            ...        ...         ...         ...    ...     ...   
12325     0.007143   0.029031   12.241717         0.0     12       1   
12326     0.000000   0.021333    0.000000         0.0     11       1   
12327     0.083333   0.086667    0.000000         0.0     11       1   
12328     0.000000   0.021053    0.000000         0.0     11       3   
12329     0.000000   0.066667    0.000000         0.0     11       1   

       TrafficType  VisitorType  Weekend  Revenue  total_duration  \
0                1            0        0        0        0.000000 

In [75]:
df_copy.fillna(1, inplace=True)

print(df_copy)

       BounceRates  ExitRates  PageValues  SpecialDay  Month  Region  \
0         0.200000   0.200000    0.000000         0.0      2       1   
1         0.000000   0.100000    0.000000         0.0      2       1   
2         0.200000   0.200000    0.000000         0.0      2       9   
3         0.050000   0.140000    0.000000         0.0      2       2   
4         0.020000   0.050000    0.000000         0.0      2       1   
...            ...        ...         ...         ...    ...     ...   
12325     0.007143   0.029031   12.241717         0.0     12       1   
12326     0.000000   0.021333    0.000000         0.0     11       1   
12327     0.083333   0.086667    0.000000         0.0     11       1   
12328     0.000000   0.021053    0.000000         0.0     11       3   
12329     0.000000   0.066667    0.000000         0.0     11       1   

       TrafficType  VisitorType  Weekend  Revenue  total_duration  \
0                1            0        0        0        0.000000 

In [81]:
df_numpy = df_copy.to_numpy()
print(df_numpy)

[[2.00000000e-01 2.00000000e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e-01 0.00000000e+00 ... 0.00000000e+00
  6.40000000e+01 1.00000000e+00]
 [2.00000000e-01 2.00000000e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 ...
 [8.33333330e-02 8.66666670e-02 0.00000000e+00 ... 0.00000000e+00
  1.84250000e+02 1.00000000e+00]
 [0.00000000e+00 2.10526320e-02 0.00000000e+00 ... 0.00000000e+00
  4.21000000e+02 8.21852732e-01]
 [0.00000000e+00 6.66666670e-02 0.00000000e+00 ... 0.00000000e+00
  2.12500000e+01 1.00000000e+00]]


In [95]:
import math
df_numpy.shape[0]
testSample = np.random.choice(df_numpy.shape[0], math.floor(0.7 * df_numpy.shape[0]), replace=False)

In [99]:
print(testSample)

[ 2537  6660  1734 ...   680 10859 11153]


In [111]:
print(df_numpy[testSample])

train = df_numpy[testSample]
test = df_numpy[~testSample]

print("Train length: " + str(train.shape[0]))
print("Test lenght: " + str(test.shape[0]))

[[1.21212100e-03 6.27705600e-03 4.50438182e+00 ... 0.00000000e+00
  8.75315873e+02 8.26348402e-01]
 [5.33333330e-02 6.44444440e-02 0.00000000e+00 ... 0.00000000e+00
  2.69400000e+02 1.00000000e+00]
 [0.00000000e+00 1.42857140e-02 0.00000000e+00 ... 0.00000000e+00
  6.66100000e+02 1.00000000e+00]
 ...
 [0.00000000e+00 2.50000000e-02 0.00000000e+00 ... 0.00000000e+00
  1.28500000e+02 1.00000000e+00]
 [0.00000000e+00 3.52941180e-02 0.00000000e+00 ... 0.00000000e+00
  1.73162500e+03 1.00000000e+00]
 [2.00000000e-02 2.00000000e-02 0.00000000e+00 ... 0.00000000e+00
  7.93125000e+02 8.49330181e-01]]
Train length: 8631
Test lenght: 8631


In [113]:
from sklearn.model_selection import train_test_split

In [145]:
df_copy_y = df_copy['Revenue']
df_copy_x = df_copy.loc[:, df_copy.columns != 'Revenue']

#print(df_copy_y)
#print(df_copy_x)

In [151]:
df_copy_x_train, df_copy_x_test, df_copy_y_train, df_copy_y_test = train_test_split(
    df_copy_x, df_copy_y, test_size=0.3, random_state=42)

In [206]:
# print("df_copy_x_train shape:", df_copy_x_train.shape)
# print(df_copy_x_train.to_numpy())

numpy_x_train = df_copy_x_train.to_numpy()
numpy_x_test = df_copy_x_test.to_numpy()
numpy_y_train = df_copy_y_train.to_numpy()
numpy_y_test = df_copy_y_test.to_numpy()

num_classes = 2

input_shape = (11, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_train = np.expand_dims(numpy_x_train, -1)
numpy_x_test = np.expand_dims(numpy_x_test, -1)
print("x_train shape:", numpy_x_train.shape)
print(numpy_x_train.shape[0], "train samples")
print(numpy_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_train = keras.utils.to_categorical(numpy_y_train, num_classes)
numpy_y_test = keras.utils.to_categorical(numpy_y_test, num_classes)

print("y_train shape: ", numpy_y_train.shape)

x_train shape: (8631, 11, 1)
8631 train samples
3699 test samples
y_train shape:  (8631, 2)


In [220]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 11, 20)         │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 220)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 220)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │           442 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [222]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5700 - loss: 5.3861 - val_accuracy: 0.8461 - val_loss: 2.4673
Epoch 2/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8239 - loss: 2.5221 - val_accuracy: 0.8461 - val_loss: 2.4692
Epoch 3/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8396 - loss: 2.4551 - val_accuracy: 0.8461 - val_loss: 2.4640
Epoch 4/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8380 - loss: 2.4650 - val_accuracy: 0.8461 - val_loss: 2.4582
Epoch 5/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8448 - loss: 2.3404 - val_accuracy: 0.8472 - val_loss: 2.4455
Epoch 6/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8381 - loss: 2.3493 - val_accuracy: 0.8472 - val_loss: 2.4167
Epoch 7/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8469 - loss: 2.1049 - val_accuracy: 0.8495 - val_loss: 2.3288
Epoch 8/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8644 - loss: 1.7937 - val_accuracy: 0.8553 - val_loss:

In [224]:
score = model.evaluate(numpy_x_test, numpy_y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.3688794672489166
Test accuracy: 0.8786158561706543
